In [1]:
from time import sleep
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
titles = pd.read_csv("songs.txt", delimiter=":", index_col="id")

In [3]:
def scrape_info(title):
    r = requests.get(f"https://www.hymnal.net/en/search/h/all/{title}")
    r.raise_for_status()

    soup = BeautifulSoup(r.text, "html.parser")

    # Get the first link item in the list-group div
    link_item = soup.select_one(".list-group a")

    # Extract the text and link number from the link item
    text = link_item.contents[0].strip() if link_item else ""
    link_number = link_item["href"].split("/")[-1] if link_item else None

    sleep(0.1)
    return {"match_title": text, "match_id": link_number}

In [4]:
scrape_info('Just one touch')

{'match_title': 'Just one touch as He moves along', 'match_id': '759'}

In [5]:
best_match = titles['title'].apply(scrape_info)

In [17]:
results = titles.reset_index().join(pd.json_normalize(best_match))

In [18]:
results

,id,title,match_title,match_id
0,1,Glory To God In The Highest,When I've run the race before me,8773
1,2,"Holy, Holy, Holy","Around Thy table, holy Lord",216
2,3,Doxology,,None
3,4,Praise Ye The Father,"O God and Father, we our praises bring",54
4,5,"Come, Thou Almighty King","Come, Thou Almighty King",3
...,...,...,...,...
715,712,Another Year Is Dawning,,None
716,713,Thanks To God,Pray to touch the throne of God,783
717,714,"Come, Let Us Anew","Your ministry, O Lord",1187
718,715,Count Your Blessings,When upon life's billows you are tempest tossed,707


In [ ]:
results.to_csv('matches.csv', index=False)